In [1]:
import pandas as pd
import os

In [2]:
for root, dirs, files in os.walk(".", topdown=False):
    if 'ipums_data_drop' in root:
        for name in files:
            print(os.path.join(root, name))
        for name in dirs:
            print(os.path.join(root, name))



./ipums_data_drop/commuter_sex_counts.csv
./ipums_data_drop/commuter_commute_time_buckets_counts.csv
./ipums_data_drop/commuter_race_counts.csv
./ipums_data_drop/commuter_age_counts.csv
./ipums_data_drop/commuter_home_ownership_counts.csv
./ipums_data_drop/commuter_income_buckets_counts.csv
./ipums_data_drop/commuter_educ_counts.csv
./ipums_data_drop/commuter_health_disabilities_counts.csv
./ipums_data_drop/commuter_origin_counts.csv


In [3]:
## How many commuters total?

commuter_time_df = pd.read_csv("./ipums_data_drop/commuter_commute_time_buckets_counts.csv",index_col=0)
commuter_time_df = commuter_time_df[commuter_time_df['YEAR']==2019]
commuter_time_df[[x for x in commuter_time_df.columns if "Minutes" in x]].sum().sum()

2671634.0

2,671,634 total commuters into Manhattan

In [4]:
distances_to_cbd = pd.read_csv("puma_distance_centroid.csv")
distances_to_cbd.rename({"distance":"distance_km"},axis=1,inplace=True)

distances_to_cbd.head(2)

,STATEFIP,state,PUMA,PUMA_NAME,lat,lon,distance_km
0,9,Connecticut,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",41.405858,-73.454362,115.529906
1,9,Connecticut,101,"Fairfield, New Canaan, Wilton, Weston & Easton...",41.208068,-73.369418,98.147659


In [5]:
origin_df = pd.read_csv("./ipums_data_drop/commuter_origin_counts.csv",index_col=0)
origin_df=origin_df[origin_df['YEAR']==2019]
origin_df=origin_df.merge(right=distances_to_cbd[['STATEFIP',"PUMA","distance_km"]],on=["STATEFIP","PUMA"]\
                         ,how='left')
origin_df[origin_df['distance_km'].isna()] #good, they all join 
origin_df.head(1)

,YEAR,TransMode,Hour,PUMA,STATEFIP,PUMA_NAME,Estimated_Commuters,distance_km
0,2019,AutoOccupants,0,100,9,"Danbury, Ridgefield, Bethel, Brookfield, New F...",69.0,115.529906


In [6]:
### How many live within 25 km?
print(origin_df[origin_df['distance_km']<=25]['Estimated_Commuters'].sum())
print("out of")
print(origin_df['Estimated_Commuters'].sum())
print()
print(100*round(origin_df[origin_df['distance_km']<=25]['Estimated_Commuters'].sum()/origin_df['Estimated_Commuters'].sum(),4),"%")

1995372.0
out of
2671634.0

74.69 %


In [7]:
### Gender Breakdown
gender_df = pd.read_csv("./ipums_data_drop/commuter_sex_counts.csv",index_col=0)
gender_df=gender_df[gender_df['YEAR']==2019]
gender_df[["Sex_F","Sex_M"]].sum()

Sex_F    1247558.0
Sex_M    1424076.0
dtype: float64

In [8]:
### Income Breakdown
income_df = pd.read_csv("./ipums_data_drop/commuter_income_buckets_counts.csv",index_col=0)
income_df = income_df[income_df['YEAR']==2019]
income_df[[x for x in income_df.columns if "income" in x] ].sum().to_clipboard() #pasted to google sheet
income_df[[x for x in income_df.columns if "income" in x] ].sum()

income_decile_0        360.0
income_decile_1     270095.0
income_decile_10    267110.0
income_decile_2     265330.0
income_decile_3     315911.0
income_decile_4     228616.0
income_decile_5     255897.0
income_decile_6     269133.0
income_decile_7     268197.0
income_decile_8     272761.0
income_decile_9     258224.0
dtype: float64

In [9]:
### Direction - how many people could use bridges that realistically could have bike infra.?
origin_df[((origin_df['PUMA_NAME'].str.contains("NYC-Brook"))|
          (origin_df['PUMA_NAME'].str.contains("NYC-Bronx"))|
         (origin_df['PUMA_NAME'].str.contains("NYC-Q")))\
         &(origin_df['distance_km']<=25)]['Estimated_Commuters'].sum()

1010495.0

In [10]:
### Age - counts by bracket
age_df = pd.read_csv("./ipums_data_drop/commuter_age_counts.csv",index_col=0)
age_df = age_df[age_df['YEAR']==2019]

print("age 16-30",(age_df[[x for x in age_df.columns if "Age_1" in x or "Age_2" in x]].sum().sum()))
print("age 30-40",(age_df[[x for x in age_df.columns if "Age_3" in x]].sum().sum()))
print("age 40-50",(age_df[[x for x in age_df.columns if "Age_4" in x]].sum().sum()))
print("age 50-60",(age_df[[x for x in age_df.columns if "Age_5" in x]].sum().sum()))
print("age 60-70",(age_df[[x for x in age_df.columns if "Age_6" in x]].sum().sum()))
print("age 70+",(age_df[[x for x in age_df.columns if "Age_7" in x or "Age_8" in x or "Age_9" in x]].sum().sum()))
      
# [x for x in age_df.columns if "Age_1" in x or "Age_2" in x]

age 16-30 622754.0
age 30-40 730600.0
age 40-50 561571.0
age 50-60 462113.0
age 60-70 242140.0
age 70+ 52456.0


In [11]:
### Access to other transit - do they take transit now and are within 25km?
origin_df[(origin_df['TransMode'].isin(["Bus","CommuterRail","Subway"]))\
         &(origin_df['distance_km']<=25)]["Estimated_Commuters"].sum()


1497410.0

In [12]:
### Walking possibility - less than 3km
origin_df[(origin_df['distance_km']<=3)]["Estimated_Commuters"].sum()

260623.0